# CSV Processing: Delete Everything
This notebook will tear down all resources created by the create_all notebook.

In [1]:
import os
import subprocess
import boto3
import json
import datetime

import pykx as kx

from managed_kx import *
from env import *

# Cluster names and database
# ----------------------------------------------------------------
DB_NAME="DEMO_DB"
DBVIEW_NAME=f"{DB_NAME}_VIEW"
SCALING_GROUP_NAME="DEMO_SCALING_GROUP"
VOLUME_NAME="DEMO_SHARED_VOLUME"
CODEBASE="demo"
CLUSTER_NAME="demo_csv_cluster"
HDB_CLUSTER_NAME="demo_hdb_cluster"

clusters=[ {'name': CLUSTER_NAME}, {'name': HDB_CLUSTER_NAME} ]
# ----------------------------------------------------------------

# Selective Deletion or All
DELETE_ALL=True
#DELETE_ALL=False

DELETE_CLUSTERS=True
DELETE_SCALING_GROUP=False
DELETE_DATAVIEW=True
DELETE_VOLUME=False
DELETE_DATABASE=True

if DELETE_ALL:
    DELETE_CLUSTERS=True
    DELETE_SCALING_GROUP=True
    DELETE_DATAVIEW=True
    DELETE_VOLUME=True
    DELETE_DATABASE=True
    

In [2]:
# Get credentials and create service client
session=None

if AWS_ACCESS_KEY_ID is None:
    print("Using Defaults ...")
    # create AWS session: using access variables
    session = boto3.Session()
else:
    print("Using variables ...")
    session = boto3.Session(
        aws_access_key_id=AWS_ACCESS_KEY_ID,
        aws_secret_access_key=AWS_SECRET_ACCESS_KEY,
        aws_session_token=AWS_SESSION_TOKEN
    )

# create finspace client
client = session.client(service_name='finspace', endpoint_url=ENDPOINT_URL)

Using Defaults ...


## Delete all resources
Using service APIs to delete all resources, if resource does not exist, assume it was already deleted and continue.

### APIs Used
[get_kx_cluster](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/finspace/client/get_kx_cluster.html)  
[delete_kx_cluster](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/finspace/client/delete_kx_cluster.html)  
[get_kx_dataview](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/finspace/client/get_kx_dataview.html)  
[delete_kx_dataview](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/finspace/client/delete_kx_dataview.html)  
[get_kx_volume](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/finspace/client/get_kx_volume.html)  
[delete_kx_volume](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/finspace/client/delete_kx_volume.html)  
[get_kx_scaling_group](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/finspace/client/get_kx_scaling_group.html)  
[delete_kx_scaling_group](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/finspace/client/get_kx_scaling_group.html)  
[get_kx_database](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/finspace/client/get_kx_database.html)  
[delete_kx_database](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/finspace/client/delete_kx_database.html)  

In [3]:
# Clusters
#-------------------------------------------------------------------------------------------
delete_status = ['RUNNING', 'CREATING', 'ACTIVE', 'CREATE_FAILED']
deleting_status = ['DELETING']

if DELETE_CLUSTERS:
    print("Deleting Clusters...")

    cluster_names = set([d['name'] for d in clusters if 'name' in d])

    # Clusters
    for c in cluster_names:
        this_c = get_kx_cluster(client=client, clusterName=c)
        if this_c is None:
            print(f"Not Found: {c}")
            continue
        elif this_c['status'] not in delete_status:
            print(f"Cannot Delete: {c} Status: {this_c['status']}")
            continue

        print(f"\tDelete: {c}")
        client.delete_kx_cluster(environmentId=ENV_ID, clusterName=c)
        print(f"\tDeleted: {c}")

    print(f"** Deleted {len(cluster_names)} Clusters **")

# Dataview
#-------------------------------------------------------------------------------------------
if DELETE_DATAVIEW:
    print()
    print(f"Deleting Dataview: {DBVIEW_NAME}...")
    this_view = get_kx_dataview(client=client, environmentId=ENV_ID, databaseName=DB_NAME, dataviewName=DBVIEW_NAME)

    if this_view is None:
        print(f"Not Found: {DBVIEW_NAME}")
    elif this_view['status'] in deleting_status:
        wait_for_dataview_status(client=client, environmentId=ENV_ID, databaseName=DB_NAME, dataviewName=DBVIEW_NAME, status='DELETED', show_wait=True)
        print("** Deleted View **")
    elif this_view['status'] not in delete_status:
        print(f"Cannot Delete: {DBVIEW_NAME} Status: {this_view['status']}")
    else:
        print(f"delete: {DBVIEW_NAME}")
        client.delete_kx_dataview(environmentId=ENV_ID, databaseName=DB_NAME, dataviewName=DBVIEW_NAME)
        wait_for_dataview_status(client=client, environmentId=ENV_ID, databaseName=DB_NAME, dataviewName=DBVIEW_NAME, status='DELETED', show_wait=True)
        print("** Deleted View **")

# Volume
#-------------------------------------------------------------------------------------------
if DELETE_VOLUME:
    print()
    print(f"Deleting Volume: {VOLUME_NAME}...")
    this_volume = get_kx_volume(client=client, environmentId=ENV_ID, volumeName=VOLUME_NAME)

    if this_volume is None:
        print(f"Not Found: {VOLUME_NAME}")
    elif this_volume['status'] in deleting_status:
        wait_for_volume_status(client=client, environmentId=ENV_ID, volumeName=VOLUME_NAME, status='DELETED', show_wait=True)
        print("** Deleted Volume **")
    elif this_volume['status'] not in delete_status:
        print(f"Cannot Delete: {VOLUME_NAME} Status: {this_volume['status']}")
    else:
        print(f"Delete: {VOLUME_NAME}")
        client.delete_kx_volume(environmentId=ENV_ID, volumeName=VOLUME_NAME)
        wait_for_volume_status(client=client, environmentId=ENV_ID, volumeName=VOLUME_NAME, status='DELETED', show_wait=True)
        print("** Deleted Volume **")

# Scaling Group
#-------------------------------------------------------------------------------------------
if DELETE_SCALING_GROUP:
    print()
    print(f"Deleting Scaling Group: {SCALING_GROUP_NAME}...")
    this_sg = get_kx_scaling_group(client=client, environmentId=ENV_ID, scalingGroupName=SCALING_GROUP_NAME)

    if this_sg is None:
        print(f"Not Found: {SCALING_GROUP_NAME}")
    elif this_sg['status'] in deleting_status:
        wait_for_scaling_group_status(client=client, environmentId=ENV_ID, scalingGroupName = SCALING_GROUP_NAME, status='DELETED', show_wait=True)
        print("** Deleted Scaling Group **")
    elif this_sg['status'] not in delete_status:
        print(f"Cannot Delete: {SCALING_GROUP_NAME} Status: {this_sg['status']}")
    else:
        print(f"Delete: {SCALING_GROUP_NAME}")
        client.delete_kx_scaling_group(environmentId=ENV_ID, scalingGroupName=SCALING_GROUP_NAME)
        wait_for_scaling_group_status(client=client, environmentId=ENV_ID, scalingGroupName = SCALING_GROUP_NAME, status='DELETED', show_wait=True)
        print("** Deleted Scaling Group **")

# Database
#-------------------------------------------------------------------------------------------
if DELETE_DATABASE:
    print()
    print(f"Deleting Database: {DB_NAME}...")
    this_db = get_kx_database(client=client, environmentId=ENV_ID, databaseName=DB_NAME)

    if this_db is None:
        print(f"Not Found: {DB_NAME}")
    else:
        print(f"delete: {DB_NAME}")
        client.delete_kx_database(environmentId=ENV_ID, databaseName=DB_NAME)    
        print("** Deleted Database **")

print("\nDone Deleting")

Deleting Clusters...
	Delete: demo_hdb_cluster
	Deleted: demo_hdb_cluster
	Delete: demo_csv_cluster
	Deleted: demo_csv_cluster
** Deleted 2 Clusters **

Deleting Dataview: DEMO_DB_VIEW...
delete: DEMO_DB_VIEW
Dataview: DEMO_DB_VIEW status is DELETING, total wait 0:00:00, waiting 30 sec ...
Dataview: DEMO_DB_VIEW status is DELETING, total wait 0:00:30, waiting 30 sec ...
Dataview: DEMO_DB_VIEW status is DELETING, total wait 0:01:00, waiting 30 sec ...
Dataview: DEMO_DB_VIEW status is DELETING, total wait 0:01:30, waiting 30 sec ...
Dataview: DEMO_DB_VIEW status is DELETING, total wait 0:02:00, waiting 30 sec ...
Dataview: DEMO_DB_VIEW status is DELETING, total wait 0:02:30, waiting 30 sec ...
Dataview: DEMO_DB_VIEW status is DELETING, total wait 0:03:00, waiting 30 sec ...
Dataview: DEMO_DB_VIEW status is DELETING, total wait 0:03:30, waiting 30 sec ...
Dataview: DEMO_DB_VIEW status is DELETING, total wait 0:04:00, waiting 30 sec ...
Dataview: DEMO_DB_VIEW status is DELETING, total wait

# Delete the directories created from extracting tarballs

In [4]:
!rm -rf demo algoseek-marketdata

In [5]:
print( f"Last Run: {datetime.datetime.now()}" )

Last Run: 2024-05-09 13:08:19.285582
